In [1]:
from STvAR_v3 import *
from STvARMetricObjects import *
from STvARFiniteDifference import *
from STvARAdvection import *
from STvARSymbolParsing import *
from STvARIndexing import*

In [2]:
DiffOrder = 4
DiffOrderKO = DiffOrder + 2
KOsigma = 0.3
dim = 3

In [3]:
gridvars = []
vartypes = []

u = stvar('u',  gridvar = True, vartype = 'state_fab')
chi = stvar('chi',  gridvar = True, vartype = 'state_fab')
psi = stvar('psi', gridvar = True, vartype = 'state_fab')
gamtilde_LL = stvarrank2('gamtilde_LL',  gridvar = True, vartype = 'state_fab', sym = 'sym01')
Atilde_LL = stvarrank2('Atilde_LL', gridvar = True, vartype = 'state_fab', sym = 'sym01')

# Initialization

In [4]:
InitString = ""

In [5]:
x_U = stvarrank1('x_U')

In [6]:
Mp = stvar('bareMp')
Mp.expr = 0.4856

Mm = stvar('bareMm')
Mm.expr = 0.4856

In [7]:
InitString += Mp.symb2expr() + Mm.symb2expr()

In [8]:
cp_U = stvarrank1('cp_U')
cp_U.expr = [4.891,0,0]

cm_U = stvarrank1('cm_U')
cm_U.expr = [-4.891,0,0]

In [9]:
InitString += cp_U.symb2expr() +cm_U.symb2expr()

In [10]:
Pp_U = stvarrank1('Pp_U')
Pp_U.expr = [0, 0.0969, 0]

Pm_U = stvarrank1('Pm_U')
Pm_U.expr = [0, -0.0969, 0]

In [11]:
InitString += Pp_U.symb2expr() + Pm_U.symb2expr()

In [12]:
Pp = stvar('Pp')
Pp.expr = sp.sqrt(Pp_U.symb[0]**2 + Pp_U.symb[1]**2 + Pp_U.symb[2]**2)

Pm = stvar('Pm')
Pm.expr = sp.sqrt(Pm_U.symb[0]**2 + Pm_U.symb[1]**2 + Pm_U.symb[2]**2)

In [13]:
InitString += Pp.symb2expr() + Pm.symb2expr()

In [14]:
xpc_U = stvarrank1('xpc_U')
xpc_U.expr = x_U.symb-cp_U.symb

xmc_U = stvarrank1('xmc_U')
xmc_U.expr = x_U.symb-cm_U.symb

In [15]:
InitString+= xpc_U.symb2expr() + xmc_U.symb2expr()

In [16]:
rpc = stvar('rpc')
rpc.expr = sp.Max(0.001, sp.sqrt(xpc_U.symb[0]**2+xpc_U.symb[1]**2+xpc_U.symb[2]**2))

rmc = stvar('rmc')
rmc.expr = sp.Max(0.001, sp.sqrt(xmc_U.symb[0]**2+xmc_U.symb[1]**2+xmc_U.symb[2]**2))

In [17]:
InitString += rpc.symb2expr()+rmc.symb2expr()

In [18]:
npc_U = stvarrank1('npc_U')
npc_U.expr = xpc_U.symb/rpc.symb

nmc_U = stvarrank1('nmc_U')
nmc_U.expr = xmc_U.symb/rmc.symb

In [19]:
InitString += npc_U.symb2expr()
InitString += nmc_U.symb2expr()

In [20]:
Abarp_UU = stvarrank2('Abarp_UU', sym = 'sym01')

for i in range(3):
    for j in range(3):
        Abarp_UU.expr[i][j] += 3/(2*rpc.symb**2)*(Pp_U.symb[i]*npc_U.symb[j]+Pp_U.symb[j]*npc_U.symb[i])
        for k in range(3):
            Abarp_UU.expr[i][j] += -3/(2*rpc.symb**2)*(sp.eye(3)[i,j]-npc_U.symb[i]*npc_U.symb[j])*npc_U.symb[k]*Pp_U.symb[k]
            
Abarm_UU = stvarrank2('Abarm_UU', sym = 'sym01')

for i in range(3):
    for j in range(3):
        Abarm_UU.expr[i][j] += 3/(2*rmc.symb**2)*(Pm_U.symb[i]*nmc_U.symb[j]+Pm_U.symb[j]*nmc_U.symb[i])
        for k in range(3):
            Abarm_UU.expr[i][j] += -3/(2*rmc.symb**2)*(sp.eye(3)[i,j]-nmc_U.symb[i]*nmc_U.symb[j])*nmc_U.symb[k]*Pm_U.symb[k]

In [21]:
InitString+=Abarp_UU.symb2expr()+Abarm_UU.symb2expr()

In [22]:
Abar_UU = stvarrank2('Abar_UU', sym='sym01')
Abar_UU.expr = Abarp_UU.symb + Abarm_UU.symb

In [23]:
InitString+=Abar_UU.symb2expr()

In [24]:
alphafunc = stvar('alphafunc', gridvar = True, vartype = 'state_fab')
alphafunc.expr = 1/(Mp.symb/(2*rpc.symb)+Mm.symb/(2*rmc.symb))

In [25]:
InitString += alphafunc.setisymb()+alphafunc.symb2expr()

In [26]:
betafunc = stvar('betafunc', gridvar = True, vartype = 'state_fab')
for i in range(3):
    for j in range(3):
        betafunc.expr += 1/8*(alphafunc.symb**7)*Abar_UU.symb[i][j]**2

In [27]:
InitString += betafunc.setisymb()

In [28]:
psi.expr = 1+1/alphafunc.symb 

In [29]:
InitString += psi.setisymb()+psi.symb2expr()

In [30]:
chi.expr = psi.symb**(-4)

In [31]:
InitString += chi.setisymb()

In [32]:
Abar_LL = stvarrank2('Abar_LL', sym = 'sym01')
Abar_LL.expr = Abar_UU.symb

In [33]:
InitString += Abar_LL.symb2expr()

In [34]:
Atilde_LL.expr = psi.symb**(-6)*Abar_LL.symb

In [35]:
InitString += Atilde_LL.setisymb()

In [36]:
up0 = stvar('up0')
up0.expr = Mp.symb**4 + 10*(Mp.symb**3)*rpc.symb + 40*(Mp.symb**2)*rpc.symb**2 + 80*Mp.symb*rpc.symb**3 + 80*rpc.symb**4

um0 = stvar('um0')
um0.expr = Mm.symb**4 + 10*(Mm.symb**3)*rmc.symb + 40*(Mm.symb**2)*rmc.symb**2 + 80*Mm.symb*rmc.symb**3 + 80*rmc.symb**4

In [37]:
InitString += up0.symb2expr() + um0.symb2expr()

In [38]:
up2 = stvar('up2')
up2.expr = (Mp.symb/(5*rpc.symb**3))*(42*(Mp.symb**5)*rpc.symb + 378*(Mp.symb**4)*rpc.symb**2 + 1316*(Mp.symb**3)*rpc.symb**3 + 2156*(Mp.symb**2)*rpc.symb**4 + 1536*Mp.symb*rpc.symb**5 + 240*rpc.symb**6 + 21*Mp.symb*((Mp.symb + 2*rpc.symb)**5)*sp.log(Mp.symb/(Mp.symb + 2*rpc.symb)))

um2 = stvar('um2')
um2.expr = (Mm.symb/(5*rmc.symb**3))*(42*(Mm.symb**5)*rmc.symb + 378*(Mm.symb**4)*rmc.symb**2 + 1316*(Mm.symb**3)*rmc.symb**3 + 2156*(Mm.symb**2)*rmc.symb**4 + 1536*Mm.symb*rmc.symb**5 + 240*rmc.symb**6 + 21*Mm.symb*((Mm.symb + 2*rmc.symb)**5)*sp.log(Mm.symb/(Mm.symb + 2*rmc.symb)))

In [39]:
InitString += up2.symb2expr() + um2.symb2expr()

In [40]:
cosp = stvar('cosp')
cosp.expr = xpc_U.symb[1]/rpc.symb

cosm = stvar('cosm')
cosm.expr = xmc_U.symb[1]/rmc.symb

In [41]:
InitString += cosp.symb2expr() + cosm.symb2expr()

In [42]:
Lp2 = stvar('Lp2')
Lp2.expr = (3/2)*cosp.symb**2 - 1/2

Lm2 = stvar('Lm2')
Lm2.expr = (3/2)*cosm.symb**2 - 1/2

In [43]:
InitString += Lp2.symb2expr() + Lm2.symb2expr()

In [44]:
uPp = stvar('uPp')
uPp.expr = Mp.symb*(Pp.symb/Mp.symb)**2/(8*(Mp.symb + 2*rpc.symb)**5)*(up0.symb+up2.symb*Lp2.symb)

uPm = stvar('uPm')
uPm.expr = Mm.symb*(Pm.symb/Mm.symb)**2/(8*(Mm.symb + 2*rmc.symb)**5)*(um0.symb+um2.symb*Lm2.symb)

In [45]:
InitString += uPp.symb2expr() + uPm.symb2expr()

In [46]:
u.expr = uPp.symb + uPm.symb

In [47]:
InitString += u.setisymb()

In [48]:
print(InitString.replace("state_fabIdx", "Idx"))

        amrex::Real bareMp = 0.48559999999999998;

        amrex::Real bareMm = 0.48559999999999998;

        amrex::Real cp_U_0 = 4.891;
        amrex::Real cp_U_1 = 0;
        amrex::Real cp_U_2 = 0;

        amrex::Real cm_U_0 = -4.891;
        amrex::Real cm_U_1 = 0;
        amrex::Real cm_U_2 = 0;

        amrex::Real Pp_U_0 = 0;
        amrex::Real Pp_U_1 = 0.0969;
        amrex::Real Pp_U_2 = 0;

        amrex::Real Pm_U_0 = 0;
        amrex::Real Pm_U_1 = -0.0969;
        amrex::Real Pm_U_2 = 0;

        amrex::Real Pp = std::sqrt(std::pow(Pp_U_0, 2) + std::pow(Pp_U_1, 2) + std::pow(Pp_U_2, 2));

        amrex::Real Pm = std::sqrt(std::pow(Pm_U_0, 2) + std::pow(Pm_U_1, 2) + std::pow(Pm_U_2, 2));

        amrex::Real xpc_U_0 = -cp_U_0 + x_U_0;
        amrex::Real xpc_U_1 = -cp_U_1 + x_U_1;
        amrex::Real xpc_U_2 = -cp_U_2 + x_U_2;

        amrex::Real xmc_U_0 = -cm_U_0 + x_U_0;
        amrex::Real xmc_U_1 = -cm_U_1 + x_U_1;
        amrex::Real xmc_U_2 = -cm_U_2 + x_U_2;

  

# Diagnostics

In [49]:
Hchi = stvar('Hchi', gridvar = True, varprefix = 'diag', vartype = 'Diag')
Hpsi = stvar('Hpsi', gridvar = True, varprefix = 'diag', vartype = 'Diag')
Hu = stvar('Hu', gridvar = True, varprefix = 'diag', vartype = 'Diag')

In [50]:
DiagString = ""

In [51]:
DiagString += u.symb2isymb() + psi.symb2isymb() + chi.symb2isymb() + alphafunc.symb2isymb() + betafunc.symb2isymb()

In [52]:
dDu_L = u.diff('dD', Accuracy = DiffOrder)
dDDu_LL = u.diff('dDD', Accuracy = DiffOrder)

dDpsi_L = psi.diff('dD', Accuracy = DiffOrder)
dDDpsi_LL = psi.diff('dDD', Accuracy = DiffOrder)

dDchi_L = chi.diff('dD', Accuracy = DiffOrder)
dDDchi_LL = chi.diff('dDD', Accuracy = DiffOrder)

In [53]:
DiagString += dDu_L.symb2isymb()+dDDu_LL.symb2isymb()
DiagString += dDpsi_L.symb2isymb()+dDDpsi_LL.symb2isymb()
DiagString += dDchi_L.symb2isymb()+dDDchi_LL.symb2isymb()

In [54]:
RsclrPsi = stvar('RsclrPsi')
RsclrChi = stvar('RsclrChi')

In [55]:
for k in range(3):
    RsclrPsi.expr += -8*(psi.symb**(-5))*dDDpsi_LL.symb[k][k]
    RsclrChi.expr += 2*dDDchi_LL.symb[k][k]-5/(2*chi.symb)*dDchi_L.symb[k]*dDchi_L.symb[k]

In [56]:
DiagString += RsclrPsi.symb2expr() + RsclrChi.symb2expr()

In [57]:
DiagString += Atilde_LL.symb2isymb()

In [58]:
Atilde_UU = stvarrank2('Atilde_UU', sym = 'sym01')
Atilde_UU.expr = Atilde_LL.symb

In [59]:
DiagString += Atilde_UU.symb2expr()

In [60]:
Hpsi.expr += RsclrPsi.symb
Hchi.expr += RsclrChi.symb
for i in range(3):
    for j in range(3):
        Hpsi.expr += Atilde_LL.symb[i][j]*Atilde_UU.symb[i][j]
        Hchi.expr += Atilde_LL.symb[i][j]*Atilde_UU.symb[i][j]

In [61]:
Hu.expr += betafunc.symb*(alphafunc.symb*(1+u.symb)+1)**(-7)
for k in range(3):
    Hu.expr += dDDu_LL.symb[k][k]
    
Hu.expr *= psi.symb**(-5)

In [62]:
DiagString += Hpsi.setisymb() + Hchi.setisymb() + Hu.setisymb()

In [63]:
print(DiagString.replace('DiagIdx', 'Diag').replace('state_fabIdx', 'Idx'))

        amrex::Real u = state_fab(i, j, k, Idx::u);

        amrex::Real psi = state_fab(i, j, k, Idx::psi);

        amrex::Real chi = state_fab(i, j, k, Idx::chi);

        amrex::Real alphafunc = state_fab(i, j, k, Idx::alphafunc);

        amrex::Real betafunc = state_fab(i, j, k, Idx::betafunc);

        amrex::Real dDu_L_0 = ((2.0/3.0)*state_fab(i + 1, j, k, Idx::u) - 1.0/12.0*state_fab(i + 2, j, k, Idx::u) - 2.0/3.0*state_fab(i - 1, j, k, Idx::u) + (1.0/12.0)*state_fab(i - 2, j, k, Idx::u))/dx[0];
        amrex::Real dDu_L_1 = ((2.0/3.0)*state_fab(i, j + 1, k, Idx::u) - 1.0/12.0*state_fab(i, j + 2, k, Idx::u) - 2.0/3.0*state_fab(i, j - 1, k, Idx::u) + (1.0/12.0)*state_fab(i, j - 2, k, Idx::u))/dx[1];
        amrex::Real dDu_L_2 = ((2.0/3.0)*state_fab(i, j, k + 1, Idx::u) - 1.0/12.0*state_fab(i, j, k + 2, Idx::u) - 2.0/3.0*state_fab(i, j, k - 1, Idx::u) + (1.0/12.0)*state_fab(i, j, k - 2, Idx::u))/dx[2];

        amrex::Real dDDu_LL_00 = ((4.0/3.0)*state_fab(i + 1, j, k, Idx::u) 

# RHS

In [64]:
RHSString = ""

In [65]:
RHSString += u.symb2isymb()

In [66]:
RHSString += dDu_L.symb2isymb()+dDDu_LL.symb2isymb()

In [67]:
RHSString += alphafunc.symb2isymb() + betafunc.symb2isymb()

In [68]:
RHSu = stvar('u', gridvar = True, addtolist = False, vartype = "state_fab", varprefix = "rhs_fab")
RHSu.expr += betafunc.symb*(alphafunc.symb*(1+u.symb)+1)**(-7)
for k in range(3):
    RHSu.expr += dDDu_LL.symb[k][k]

In [69]:
RHSString += RHSu.setisymb()

In [70]:
print(RHSu.isymb)

rhs_fab[i, j, k, state_fabu]


In [71]:
print(RHSString.replace("state_fabIdx", "Idx"))

        amrex::Real u = state_fab(i, j, k, Idx::u);

        amrex::Real dDu_L_0 = ((2.0/3.0)*state_fab(i + 1, j, k, Idx::u) - 1.0/12.0*state_fab(i + 2, j, k, Idx::u) - 2.0/3.0*state_fab(i - 1, j, k, Idx::u) + (1.0/12.0)*state_fab(i - 2, j, k, Idx::u))/dx[0];
        amrex::Real dDu_L_1 = ((2.0/3.0)*state_fab(i, j + 1, k, Idx::u) - 1.0/12.0*state_fab(i, j + 2, k, Idx::u) - 2.0/3.0*state_fab(i, j - 1, k, Idx::u) + (1.0/12.0)*state_fab(i, j - 2, k, Idx::u))/dx[1];
        amrex::Real dDu_L_2 = ((2.0/3.0)*state_fab(i, j, k + 1, Idx::u) - 1.0/12.0*state_fab(i, j, k + 2, Idx::u) - 2.0/3.0*state_fab(i, j, k - 1, Idx::u) + (1.0/12.0)*state_fab(i, j, k - 2, Idx::u))/dx[2];

        amrex::Real dDDu_LL_00 = ((4.0/3.0)*state_fab(i + 1, j, k, Idx::u) - 1.0/12.0*state_fab(i + 2, j, k, Idx::u) + (4.0/3.0)*state_fab(i - 1, j, k, Idx::u) - 1.0/12.0*state_fab(i - 2, j, k, Idx::u) - 5.0/2.0*state_fab(i, j, k, Idx::u))/std::pow(dx[0], 2);
        amrex::Real dDDu_LL_01 = ((4.0/9.0)*state_fab(i + 1, j + 

# Post Update

In [72]:
PUString = ""

In [73]:
PUString += u.symb2isymb()

In [74]:
PUString += Mp.symb2expr() + Mm.symb2expr()

In [75]:
PUString += cp_U.symb2expr() +cm_U.symb2expr()

In [76]:
PUString += Pp_U.symb2expr() + Pm_U.symb2expr()

In [77]:
PUString+= xpc_U.symb2expr() + xmc_U.symb2expr()

In [78]:
PUString += rpc.symb2expr()+rmc.symb2expr()

In [79]:
PUString += npc_U.symb2expr()+nmc_U.symb2expr()

In [80]:
PUString+=Abarp_UU.symb2expr()+Abarm_UU.symb2expr()

In [81]:
PUString+=Abar_UU.symb2expr()

In [82]:
PUString += alphafunc.setisymb()
PUString += alphafunc.symb2expr()

In [83]:
PUString += betafunc.setisymb()

In [84]:
psi.expr += u.symb

In [85]:
PUString += psi.setisymb()
PUString += psi.symb2expr()

In [86]:
PUString += chi.setisymb()

In [87]:
PUString += Abar_LL.symb2expr()

In [88]:
PUString += Atilde_LL.setisymb()

In [89]:
print(PUString.replace("state_fabIdx", "Idx"))

        amrex::Real u = state_fab(i, j, k, Idx::u);

        amrex::Real bareMp = 0.48559999999999998;

        amrex::Real bareMm = 0.48559999999999998;

        amrex::Real cp_U_0 = 4.891;
        amrex::Real cp_U_1 = 0;
        amrex::Real cp_U_2 = 0;

        amrex::Real cm_U_0 = -4.891;
        amrex::Real cm_U_1 = 0;
        amrex::Real cm_U_2 = 0;

        amrex::Real Pp_U_0 = 0;
        amrex::Real Pp_U_1 = 0.0969;
        amrex::Real Pp_U_2 = 0;

        amrex::Real Pm_U_0 = 0;
        amrex::Real Pm_U_1 = -0.0969;
        amrex::Real Pm_U_2 = 0;

        amrex::Real xpc_U_0 = -cp_U_0 + x_U_0;
        amrex::Real xpc_U_1 = -cp_U_1 + x_U_1;
        amrex::Real xpc_U_2 = -cp_U_2 + x_U_2;

        amrex::Real xmc_U_0 = -cm_U_0 + x_U_0;
        amrex::Real xmc_U_1 = -cm_U_1 + x_U_1;
        amrex::Real xmc_U_2 = -cm_U_2 + x_U_2;

        amrex::Real rpc = std::max(0.00100000000000000, std::sqrt(std::pow(xpc_U_0, 2) + std::pow(xpc_U_1, 2) + std::pow(xpc_U_2, 2)));

        amrex::R